parameter tune
- tour terbanyak
- day
- time 


experiment
- tour terbanyak / epoch
- day / epoch
- time / epoch


In [ ]:
import numpy as np
from pprint import pprint
from copy import deepcopy
from utils.base_model import BaseModel
from mealpy.swarm_based.PSO import OriginalPSO

In [ ]:
## Setting parameters
base_params = {
    "model": OriginalPSO,
    "maut_weights" : {
        "popular" : 1,
        "cost" : 1,
        "time" : 1
    },
    "time_start": 8,
    "time_end": 20,
    
    "n_days": 50,
    "hotel_id": 5,
    # "tour_ids": [0, 2, 4, 6, 7, 8, 9, 20, 23, 25, 29, 30, 31, 32, 33, 35, 40, 55, 42, 56],
    "tour_ids": list(range(0, 100)),
    
    "minmax": "max",
    "log_to": "console",
}
model_params = {
    "epoch": 30,           # iteration
    "pop_size": 50,        # population size
    "c1": 2.05,
    "c2": 2.05,
    "w_min": 0.4,
    "w_max": 0.9,
}
model = BaseModel(base_params=base_params, model_params=model_params)

In [ ]:
# ----Tuning
model_params_tune = {
    # "epoch": [15, 30, 50],
    "pop_size": [25, 50, 100],
    "c1": 2.05,
    "c2": 2.05,
    "w_min": 0.4,
    "w_max": 0.9,
}
num_test = 5
log_directory = "logs/params"
file_name = "pso"
model.params_tuner(
    model_params_tuning=model_params_tune,
    num_test=num_test,
    log_directory=log_directory,
    file_name=file_name
)
model.params_tuner(model_params_tune)

In [ ]:
# Try to Train with new best Params or default params
model_params = {
    "epoch": 30,           # iteration
    "pop_size": 50,        # population size
    "c1": 2.05,
    "c2": 2.05,
    "w_min": 0.4,
    "w_max": 0.9,
}
model.train(model_params)

In [ ]:
print(f"Count Node: {len(np.hstack([x[0] for x in model.best_solution]))}")
for idx, day in enumerate(model.best_solution):
    print("\nDay", idx+1)
    total_time = 8*3600
    tour_nodes = day[0]
    for i in range(len(tour_nodes)):
        time = 0
        spend_time = 0
        if i == 0:
            time = model.conn.get_hotel_dist_matrix(origin_id=model.hotel_node.id, dest_id=tour_nodes[i].id, hotel2tour=True)
            spend_time = 0
        elif i+1 == len(tour_nodes):
            spend_time = tour_nodes[i].spend_time
            time = model.conn.get_hotel_dist_matrix(origin_id=model.hotel_node.id, dest_id=tour_nodes[i].id, hotel2tour=False)+spend_time
            print("To Hotels -", 20*3600, int(np.floor((total_time+time)/3600)))
        else:
            spend_time = tour_nodes[i].spend_time
            time = model.conn.get_tour_dist_matrix(origin_id=tour_nodes[i].id, dest_id=tour_nodes[i+1].id)+spend_time
        total_time += time
        
        jam_berangkat = f"{int(np.floor((total_time-(time-spend_time))/3600))}:{int(np.round(((total_time-(time-spend_time))%3600)/60))}"
        jam_sampai = f"{int(np.floor(total_time/3600))}:{int(np.round((total_time%3600)/60))}"
        print(tour_nodes[i].id, tour_nodes[i].rating, tour_nodes[i].tarif, time, (time-spend_time), tour_nodes[i].jam_buka, tour_nodes[i].jam_tutup, total_time, jam_berangkat, jam_sampai)

    print(np.round(total_time/3600))
    
if len(model.outlier_solution) != 0:
    print("\nOutlier")
    pprint(model.outlier_solution)

# Experiment

In [ ]:
model_params = {
    "epoch": 30,           # iteration
    "pop_size": 50,        # population size
    "c1": 2.05,
    "c2": 2.05,
    "w_min": 0.4,
    "w_max": 0.9,
}
maut_weights = {
    "popular" : 1,
    "cost" : 1,
    "time" : 1
}
num_test_epoch_list = [10, 15, 25, 50, 75, 100, 150, 250, 500, 750]
num_test = 5
log_directory = "logs/experiment"
file_name = "pso_weight_p1_c1_t1"
model.experiment(
    model_params=model_params,
    maut_weights=maut_weights,
    num_test_epoch_list=num_test_epoch_list,
    num_test=num_test,
    log_directory=log_directory,
    file_name=file_name
)